## 데이터셋 관련
- 실습 데이터(GPT 통해 생성) : https://huggingface.co/datasets/shangrilar/ko_text2sql
- 게임 데이터베이스 맞게 형성

In [1]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you h

In [2]:
import os

### 데이터 로딩
- 영어 text2sql : https://huggingface.co/datasets/Salesforce/wikisql
- 한글어 관련 데이터 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71351

In [3]:
from datasets import load_dataset
# https://huggingface.co/datasets/shangrilar/ko_text2sql/viewer/origin
hf_dataset_text2sql = load_dataset('shangrilar/ko_text2sql', 'origin')
hf_dataset_text2sql

Generating train split:   0%|          | 0/38246 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['db_id', 'context', 'question', 'answer'],
        num_rows: 38246
    })
    test: Dataset({
        features: ['db_id', 'context', 'question', 'answer'],
        num_rows: 112
    })
})

In [4]:
# huggingface dataset to pandas DataFrame with train
df_dataset_text2sql_train = hf_dataset_text2sql['train'].to_pandas()
df_dataset_text2sql_train.dropna(inplace = True)
df_dataset_text2sql_train = df_dataset_text2sql_train.query('db_id != 1') # 정상 query 사용
df_dataset_text2sql_train.head(3)

,db_id,context,question,answer
2388,2,"CREATE TABLE messages (\n ""message_id"" SERI...",messages 테이블에서 모든 데이터를 조회해줘,SELECT * FROM messages;
2389,2,"CREATE TABLE messages (\n ""message_id"" SERI...",보낸 사람의 ID가 15보다 큰 메시지들의 수를 알려줘,SELECT COUNT(*) FROM messages WHERE sender_id ...
2390,2,"CREATE TABLE messages (\n ""message_id"" SERI...",읽지 않은 메시지들은 몇 개인가요?,SELECT COUNT(*) FROM messages WHERE read = FALSE;


In [5]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.
### DDL:
{ddl}
### Question:
{question}
### SQL:
{query}"""
    return prompt

In [6]:
# 필요한 컬럼 추출해 프롬프트 명령어 만들기
from tqdm import tqdm

df_total_count = len(df_dataset_text2sql_train)
for idx, row in tqdm(df_dataset_text2sql_train.iterrows(), total=df_total_count, desc='Generating prompts') : # DataFrame row 단위 loop
    # make_prompt(ddl, question, query='')
    prompt_command = make_prompt(ddl= row['context']
                                , question = row['question']
                                , query = row['answer'])
    df_dataset_text2sql_train.loc[idx, 'prompt'] = prompt_command
    pass

df_dataset_text2sql_train.head(3)

Generating prompts: 100%|██████████| 33876/33876 [00:06<00:00, 4852.25it/s]


,db_id,context,question,answer,prompt
2388,2,"CREATE TABLE messages (\n ""message_id"" SERI...",messages 테이블에서 모든 데이터를 조회해줘,SELECT * FROM messages;,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2389,2,"CREATE TABLE messages (\n ""message_id"" SERI...",보낸 사람의 ID가 15보다 큰 메시지들의 수를 알려줘,SELECT COUNT(*) FROM messages WHERE sender_id ...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2390,2,"CREATE TABLE messages (\n ""message_id"" SERI...",읽지 않은 메시지들은 몇 개인가요?,SELECT COUNT(*) FROM messages WHERE read = FALSE;,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [7]:
!mkdir ./datasets

In [8]:
# 재사용 위한 저장
dataset_path = 'datasets'
df_dataset_text2sql_train.to_csv(f'{dataset_path}/train.csv')

In [9]:
# huggingface dataset to pandas DataFrame with train test
df_dataset_text2sql_test = hf_dataset_text2sql['test'].to_pandas()
df_dataset_text2sql_test.head(3)

,db_id,context,question,answer
0,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,각 보상 아이템별로 보상 경험치의 합을 구해줘,"SELECT reward_items, SUM(reward_experience) AS..."
1,1,CREATE TABLE players (\n player_id INT PRIMAR...,사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.,SELECT COUNT(*) FROM players WHERE username LI...
2,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,퀘스트 진행 상황이 100%인 퀘스트의 이름과 보상 경험치는 얼마인가요?,"SELECT q.name, q.reward_experience FROM quests..."


In [10]:
# 필요한 컬럼 추출해 프롬프트 명령어 만들기
from tqdm import tqdm

df_total_count = len(df_dataset_text2sql_test)
for idx, row in tqdm(df_dataset_text2sql_test.iterrows(), total=df_total_count, desc='Generating prompts') : # DataFrame row 단위 loop
    # make_prompt(ddl, question, query='')
    prompt_command = make_prompt(ddl= row['context']
                                , question = row['question']
                                , query = '')
    df_dataset_text2sql_test.loc[idx, 'prompt'] = prompt_command
    pass

df_dataset_text2sql_test.head(3)

Generating prompts: 100%|██████████| 112/112 [00:00<00:00, 3382.75it/s]


,db_id,context,question,answer,prompt
0,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,각 보상 아이템별로 보상 경험치의 합을 구해줘,"SELECT reward_items, SUM(reward_experience) AS...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
1,1,CREATE TABLE players (\n player_id INT PRIMAR...,사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.,SELECT COUNT(*) FROM players WHERE username LI...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,퀘스트 진행 상황이 100%인 퀘스트의 이름과 보상 경험치는 얼마인가요?,"SELECT q.name, q.reward_experience FROM quests...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [11]:
df_dataset_text2sql_test.to_csv(f'{dataset_path}/test.csv')

## linux 명령어 알기

In [12]:
!ls -la
# !ls -la ../
# !pwd

total 76
drwxr-xr-x 3 root root  4096 Feb 10 03:29 .
drwxr-xr-x 6 root root  4096 Feb 10 03:29 ..
drwxr-xr-x 2 root root  4096 Feb 10 03:29 datasets
---------- 1 root root 64170 Feb 10 03:29 __notebook__.ipynb


In [13]:
!zip -r GenerativeLLMwithHuggingface_Datasets.zip *

  adding: datasets/ (stored 0%)
  adding: datasets/test.csv (deflated 92%)
  adding: datasets/train.csv (deflated 95%)
  adding: __notebook__.ipynb (deflated 89%)
